In [56]:
from myhdl import block, delay, always_seq, instance, always, Signal, ResetSignal, traceSignals
from dataclasses import dataclass
from itertools import product
from typing import Callable
from copy import deepcopy

@dataclass
class HighLevelAddressStream:
    index_generator: Callable
    

@dataclass
class LowLevelAddressStream:
    iteration_domain: product
    access_map: Callable
    condition: Callable
    done: bool = False

    def __post_init__(self):
        self.initial_iteration_domain = deepcopy(self.iteration_domain)

    def reset(self):
        self.iteration_domain = deepcopy(self.initial_iteration_domain)

    def __iter__(self):
        return self

    def __next__(self):
        try:
            next_iteration_vector = next(self.iteration_domain)
            print(next_iteration_vector)
            if self.condition(next_iteration_vector):
                next_index = self.access_map(next_iteration_vector)
            else:
                next_index = 0
        except StopIteration:
            print("Warning: stream with {} has concluded".format(self))
            next_index = 0
            self.done = True
        return next_index

    @block
    def generator_inst(self, clk, enable, reset, stream_out):
        @always(clk.posedge, reset.posedge)
        def output_stream():
            if not reset and enable:
                if not self.done:
                    stream_out.next = next(self)
            elif reset:
                self.reset()
        return output_stream

@ block
def counter(clk, reset, count):
    @always_seq(clk.posedge, reset=reset)
    def increment():
        count.next = count.val + 1
    return increment


@block
def clk_driver(clk, enable, period=20):
    lowTime = int(period / 2)
    highTime = period - lowTime

    @instance
    def drive_clk():
        while True:
            if not enable:
                yield enable
            yield delay(lowTime)
            clk.next = 1
            yield delay(highTime)
            clk.next = 0

    return drive_clk


@block
def top():
    stream = LowLevelAddressStream(product(*([range(10)]*2)), lambda iter_vect: iter_vect[0]
                           *10+iter_vect[1], lambda iter_vect: iter_vect[0] >= 0 and iter_vect[1] >= 0)
    stream_out = Signal(0)
    clk = Signal(bool(0))
    enable = Signal(bool(0))
    count = Signal(0)
    reset = ResetSignal(bool(0), active=1, isasync=True)
    counter_inst = counter(clk, reset, count)
    clk_driver_inst = clk_driver(clk, enable, period=10)
    stream_generator = stream.generator_inst(clk, enable, reset, stream_out)

    @instance
    def start_sim():
        # reset cycle
        enable.next = 0
        reset.next = 1
        yield delay(10)
        enable.next = 1
        reset.next = 0
        yield delay(40)
        enable.next = 0
        reset.next = 1
        yield delay(10)
        enable.next = 1
        reset.next = 0
        
    return clk_driver_inst, counter_inst, start_sim, stream_generator


In [57]:
inst = traceSignals(top())
inst.run_sim(100)
inst.quit_sim()

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 0)
(0, 1)
(0, 2)
(0, 3)


<class 'myhdl._SuspendSimulation'>: Simulated 100 timesteps


In [69]:
def fn(ub, lb, step):
    for i in range(lb,ub,step):
        yield i

In [70]:
t = fn(10)

In [71]:
list(t)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]